In [1]:
!pip install tensorflow_model_optimization tf_keras image_classifiers
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import tensorflow_model_optimization as tfmot
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import tf_keras as keras
from tf_keras.applications import ResNet50V2
from tf_keras.datasets import cifar100
from tf_keras import Sequential, Input
from tf_keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from tf_keras.applications.resnet_v2 import preprocess_input
from tf_keras.losses import CategoricalCrossentropy
from tf_keras.activations import linear
from tf_keras.models import Model
from tf_keras.backend import clear_session
from tf_keras.optimizers import SGD
from tf_keras.utils import Progbar
from tensorflow.nn import softmax_cross_entropy_with_logits
from classification_models.models_factory import ModelsFactory
from tensorflow_model_optimization.python.core.sparsity.keras.pruning_wrapper import PruneLowMagnitude
import os
import zipfile
import tempfile

2024-05-02 13:12:37.916195: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-02 13:12:37.916890: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 13:12:37.921643: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 13:12:37.984601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 13:12:38.921936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
class KerasModelsFactory(ModelsFactory):
    @staticmethod
    def get_kwargs():
        return {
            'backend': keras.backend,
            'layers': keras.layers,
            'models': keras.models,
            'utils': keras.utils,
        }
# Pour faire marcher image-classifiers avec tf_keras, la version compatibilité de tf.keras, nécéssaire pour tflite
Classifiers = KerasModelsFactory()

In [3]:
n_epoch = 2
batch_size = 100
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 86.25 MiB/s 1s

In [5]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [6]:
!mc cp s3/afeldmann/projet_cnam/model_etudiant_t3_a50.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras
model_etudiant = new_modele_resnet()
model_etudiant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras")
model_etudiant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

..._a50.keras: 43.54 MiB / 43.54 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 64.13 MiB/s 0s

In [7]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def load_cifar_train():
    train_dataset, _ = cifar100.load_data()
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(n_images).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset

_, test_dataset = cifar100.load_data()
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [8]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

def get_regularisation_pruning(model):
    return [(getattr(layer.layer,reg),layer.layer.kernel) for layer in model.layers if hasattr(layer, "layer") for reg in ["kernel_regularizer", "bias_regularizer"] if hasattr(layer.layer, reg) and getattr(layer.layer,reg) is not None]

@tf.function
def perte_regularisation(regularisation):
    return tf.add_n([tf.reduce_sum(reg(kernel)) for reg, kernel in regularisation])

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

kce = keras.losses.CategoricalCrossentropy()

# Pour une raison non-identifiée, tf.nn.crossentropy_with_logits se comporte mal pendant le pruning et fait baisser l'accuracy.
@tf.function
def ce(x, y_logits, temp):
    return kce(x, softmax(y_logits,temp)) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy_etu, accuracy_dis\n")
def append_csv_log(fichier, epoch, accuracy_etu, accuracy_dis):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy_etu:.2f},{accuracy_dis:.2f}\n")

def forward_backward_pass_impl(train_dataset_iter, model_etudiant_pruning, alpha, temp, optim, regularisation_etudiant):
    X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
    with tf.GradientTape() as tape:
        etudiant_estim_logit = model_etudiant_pruning(X_batch, training = True)
        perte = alpha * ce(y_batch,etudiant_estim_logit,1) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp) + perte_regularisation(regularisation_etudiant)
    grads = tape.gradient(perte, model_etudiant_pruning.trainable_variables)
    optim.apply_gradients(zip(grads, model_etudiant_pruning.trainable_variables))
    return compte_bons(etudiant_estim_logit,y_batch), compte_bons(etudiant_estim_logit, enseignant_estim_softmax)

def distillateur_kl_pruning(model_etudiant, enseignant, train_dataset, temp, nom_modele, n_epoch, alpha):
    etudiant_logit_model = get_modele_logits(model_etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    end_step = np.ceil(n_images / batch_size).astype(np.int32) * n_epoch
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                                   final_sparsity=0.70,
                                                                   begin_step=0,
                                                                   end_step=end_step)
    }
    model_etudiant_pruning = tfmot.sparsity.keras.prune_low_magnitude(etudiant_logit_model, **pruning_params)
    model_etudiant_pruning.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
    optim = model_etudiant_pruning.optimizer
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    regularisation_etudiant = get_regularisation_pruning(model_etudiant_pruning)
    forward_backward_pass = tf.function(forward_backward_pass_impl)
    step_callback = tfmot.sparsity.keras.UpdatePruningStep()
    step_callback.set_model(model_etudiant_pruning)
    step_callback.on_train_begin()
    # La tf.function ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        step_callback.on_epoch_begin(epoch)
        n_batch = n_images//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc (etu, train)", "acc (dis, train)"])
        bons_epoque_etu, bons_epoque_dis = 0, 0
        for i in range(n_batch):
            step_callback.on_train_batch_begin(i)
            bons_etu, bons_dis = forward_backward_pass(train_dataset_iter, model_etudiant_pruning, alpha, temp, optim, regularisation_etudiant)
            bons_epoque_etu += bons_etu.numpy()
            bons_epoque_dis += bons_dis.numpy()
            n_observ = (i+1) * batch_size
            accuracy_etu, accuracy_dis = bons_epoque_etu / n_observ, bons_epoque_dis / n_observ
            barre_progression.update(i + 1, values = [("acc (etu, train)", accuracy_etu), ("acc (dis, train)", accuracy_dis)])
            step_callback.on_train_batch_end(i)
        step_callback.on_epoch_end(epoch)
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy_etu, accuracy_dis)
    step_callback.on_train_end()

In [9]:
train_dataset = load_cifar_train()
nom_modele =  f"model_etudiant_pruning"
distillateur_kl_pruning(model_etudiant, model_enseignant, train_dataset, 1, nom_modele, n_epoch, 1)
wd = os.getcwd()
model_etudiant.save(f"{wd}/sauvegardes/{nom_modele}.keras")
os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

C'est parti pour la distillation !

Époque 1 / 2
 79/500 [===>..........................] - ETA: 18:39 - acc (etu, train): 0.6739 - acc (dis, train): 0.6657

KeyboardInterrupt: 

In [14]:
model_etudiant.compile(metrics="accuracy")
model_etudiant.evaluate(test_dataset)
model_etudiant.evaluate(test_dataset.map(lambda images, labels: (images,model_enseignant(images, training = False)), num_parallel_calls = tf.data.AUTOTUNE))

100/100 [==============================] - 32s 224ms/step - loss: 6.7061 - accuracy: 0.7360


KeyboardInterrupt: 

In [ ]:
distillateur_kl_pruning(model_etudiant_pruning, model_enseignant, train_dataset, 3, nom_modele, n_epoch, 0.5)

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_etudiant_pruning)
pruned_keras_file = "/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.h5"
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_tflite_model = converter.convert()
pruned_tflite_file = "/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.tflite"
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)


In [ ]:
test_batches = test_dataset.map(lambda images, labels: (images,labels,model_enseignant(images, training = False)), num_parallel_calls = tf.data.AUTOTUNE).unbatch().batch(1)

interpreter = tf.lite.Interpreter(model_path=pruned_tflite_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

test_etupreds, test_labels, test_enspreds = [], [], []
for img, label, enspred in tqdm(test_batches.take(1000)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    test_etupreds.append(interpreter.get_tensor(output_index))
    test_labels.append(label.numpy()[0])
    test_enspreds.append(enspred.numpy()[0])

score_dur, score_dis = 0, 0
for item in range(0,len(test_etupreds)):
    etupred = np.argmax(test_etupreds[item])
    label = np.argmax(test_labels[item])
    enspred = np.argmax(test_enspreds[item])
    if etupred == label:
        score_dur += 1
    if etupred == enspred:
        score_dis += 1

print(f"{score_dur} prédictions correctes sur 1000 face aux étiquettes dures et {score_dis} face aux étiquettes douces.")

In [ ]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)

In [ ]:
print("Taille du modèle enseignant zippé : %.2f bytes" % (get_gzipped_model_size("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")))
print("Taille du modèle étudiant zippé : %.2f bytes" % (get_gzipped_model_size("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras")))
print("Taille du modèle étudiant zippé après élagage : %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Taille du modèle étudiant zippé après élagage et quantification : %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

In [ ]:
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.tflite s3/afeldmann/projet_cnam/modele_etudiant_pruning.tflite
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.h5 s3/afeldmann/projet_cnam/modele_etudiant_pruning.h5